In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/slh/icecube/')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
from icecube.fastai_fix import *
from tqdm.notebook import tqdm
from icecube.data_train import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader
from icecube.loss import loss, loss_vms
from icecube.models import EncoderWithDirectionReconstructionV11_V2_LOCAL_GLOBAL
from fastxtend.vision.all import EMACallback

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-03-21 11:36:52 - get_logger - Writing log to logs/graphnet_20230321-113652.log


In [3]:
OUT = 'hb_localattentionv4FT2SWA'
PATH = '../data/'

NUM_WORKERS = 24
SEED = 2023
bs = 1024 - 256 - 128
L = 256

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
os.makedirs(OUT, exist_ok=True)

In [4]:
def WrapperAdamW(param_groups,**kwargs):
    return OptimWrapper(param_groups,torch.optim.AdamW)



In [5]:
fname = OUT

ds_train = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=4, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))

data = DataLoaders(dl_train,dl_val)
model = EncoderWithDirectionReconstructionV11_V2_LOCAL_GLOBAL()
model.load_state_dict(torch.load('/opt/slh/icecube/hb_training_loop/hb_localattentionv4FT2/models/model_7.pth'))
model = model.cuda()
model = nn.DataParallel(model)
learn = Learner(data, model,  path = OUT, loss_func=loss_vms,cbs=[GradientClip(3.0),CSVLogger(),EMACallback(),
            SaveModelCallback(monitor='loss',comp=np.less,every_epoch=True),
            GradientAccumulation(n_acc=4)],
            metrics=[loss], opt_func=partial(WrapperAdamW,eps=1e-7)).to_fp16()




In [ ]:
learn.fit_one_cycle(8, 
                     lr_max=1e-5,
                     wd=0.05, 
                     pct_start=0.01,
                     div=25,
                     div_final=25,
                     moms=(0.95,0.95,0.95))

epoch,train_loss,valid_loss,loss,time
0,1.286470,1.347106,0.979002,3:52:53
1,1.281372,1.344351,0.978752,3:53:24
2,1.297708,1.344049,0.978691,3:53:38
3,1.242978,1.344013,0.978576,3:54:20
4,1.283216,1.343200,0.978486,3:54:28
5,1.292499,1.343686,0.978401,3:54:40
6,1.255610,1.343654,0.978319,3:54:38


/opt/slh/icecube/icecube/data_train.py:253: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)
/opt/conda/lib/python3.7/site-packages/torch/cuda/nccl.py:51: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(inputs, collections.Container) or isinstance(inputs, torch.Tensor):
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_

In [ ]:
learn.validate()

In [ ]:
#learn.save('TEMP')